In [ ]:
!pip install --upgrade --quiet  gigachat
!pip install langchain

In [95]:
import pandas as pd
#from langchain.chat_models import ChatOpenAI


import sklearn
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import model_selection, preprocessing,svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier



#from langchain_community.chat_models import GigaChat
#from langchain_core.messages import HumanMessage, SystemMessage
from getpass import getpass
from sklearn.pipeline import Pipeline
from sklearn import metrics



import os

import re

import spacy

In [ ]:
train_data = pd.read_csv("/kaggle/input/train-secondpilot/train_SecondPilot/train_data.csv") 
train_data

In [ ]:
answer_class = pd.read_csv("/kaggle/input/train-secondpilot/train_SecondPilot/answer_class.csv")
answer_class

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/train-secondpilot/train_SecondPilot/sample_submission.csv")


In [ ]:
sns.histplot(data=train_data, x='answer_class', binwidth=1)


In [ ]:
sns.histplot(data=train_data, x='Category', binwidth=1)


In [ ]:
os.environ["GIGACHAT_CREDENTIALS"] = getpass()
        
        

In [ ]:
chat = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [ ]:
a = train_data['answer_class'].value_counts() 
for i in range(30):
    for j in range(3-a[i]):
        ans = answer_class.loc[i]["Answer"]
        
        messages = [
            HumanMessage(
                content=f"Придумай вопрос от первого лица, ответ на который будет: {ans}"
                ),
        ]

        print (chat.invoke(messages).content)
        
        line = {"Question": chat.invoke(messages).content, "Category": train_data.loc[i]["Category"], "answer_class": i}
        train_data = train_data._append(line, ignore_index = True)
        
        
        
        

In [ ]:
a = train_data['answer_class'].value_counts() 
a

In [ ]:
for i in range (30):
    ans = answer_class.loc[i]["Answer"]

    messages = [
        HumanMessage(
            content=f"Придумай вопрос от первого лица, ответ на который будет: {ans}"
            ),
    ]

    print (chat.invoke(messages).content)
        
    line = {"Question": chat.invoke(messages).content, "Category": train_data.loc[i]["Category"], "answer_class": i}
    train_data = train_data._append(line, ignore_index = True)
    

In [ ]:
chat2 = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [ ]:
for i in range (train_data.shape[0]):
    question = train_data.loc[i]["Question"]
    messages = [
        HumanMessage(
            content=f"Перепиши вопрос иначе вопрос: {question}"
            ),
    ]

    print (chat2.invoke(messages).content)
        
    line = {"Question": chat.invoke(messages).content, "Category": train_data.loc[i]["Category"], "answer_class": train_data.loc[i]["answer_class"]}
    train_data = train_data._append(line, ignore_index = True)
    
    

In [ ]:
train_data.Question[238]

In [ ]:
train_data.to_csv("new_data.csv",index=False )

In [ ]:
a = train_data['answer_class'].value_counts() 
a

In [ ]:
train_data

In [ ]:
le = LabelEncoder()

label = le.fit_transform(train_data['Category'])
label

In [ ]:
train_data['Category_int'] = label

train_data.drop('Category', axis=1, inplace=True)
train_data


In [ ]:
train_data = pd.merge(train_data, answer_class, on='answer_class')
train_data

In [ ]:
train_data.to_csv("train.csv",index=False)

# 1 Вариант

In [118]:
train_data = pd.read_csv("/kaggle/input/new-train-data/- 1.csv")
train_data

,Unnamed: 0,Question,answer_class,Category_int,Answer
0,NaN,"Что делать, если я хочу изменить группу или сп...",11,3.0,Мы можем перевести вас в другую группу в рамка...
1,NaN,Из чего состоит основное обучение в личном каб...,15,4.0,"Подготовка — курсы, которые помогут подготовит..."
2,NaN,Что входит в программу помощи с трудоустройств...,27,9.0,Карьерный план\nПостроите свою стратегию поиск...
3,ываы,"Подскажите расписание каникул в 2024, пожалуйста",21,7.0,Расписание каникул на 2024:\n\n26 декабря 2023...
4,NaN,Какие инструменты и ресурсы вы предоставляете ...,25,9.0,Мы помогаем нашим выпускникам найти работу. Ка...
...,...,...,...,...,...
295,NaN,После чего можно будет получить диплом?,2,NaN,NaN
296,NaN,"Что сделать, чтобы получить итоговый документ ...",1,NaN,NaN
297,NaN,Какие условия получения итогового документа об...,1,NaN,NaN
298,NaN,Какой документ подтверждает уровень компетенций?,0,NaN,NaN


In [ ]:
train_data.loc[train_data['answer_class'] == 27]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vect = TfidfVectorizer(max_features=5000,stop_words='english')

In [97]:
question = train_data.Question
question

0      Что делать, если я хочу изменить группу или сп...
1      Из чего состоит основное обучение в личном каб...
2      Что входит в программу помощи с трудоустройств...
3       Подскажите расписание каникул в 2024, пожалуйста
4      Какие инструменты и ресурсы вы предоставляете ...
                             ...                        
295              После чего можно будет получить диплом?
296    Что сделать, чтобы получить итоговый документ ...
297    Какие условия получения итогового документа об...
298     Какой документ подтверждает уровень компетенций?
299    Как подтвержрить уровень компетенций после про...
Name: Question, Length: 300, dtype: object

In [98]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text) 
    text = re.sub(r'\w*\d+\w*', '', text) 
    text = re.sub('\s{2,}', "", text) 
    return text

In [121]:
question = question.apply(clean_text)
question

0      что делать если я хочу изменить группу или спе...
1      из чего состоит основное обучение в личном каб...
2      что входит в программу помощи с трудоустройств...
3              подскажите расписание каникул впожалуйста
4      какие инструменты и ресурсы вы предоставляете ...
                             ...                        
295               после чего можно будет получить диплом
296    что сделать чтобы получить итоговый документ о...
297    какие условия получения итогового документа об...
298      какой документ подтверждает уровень компетенций
299    как подтвержрить уровень компетенций после про...
Name: Question, Length: 300, dtype: object

In [119]:
question.drop_duplicates()

0      что делать если я хочу изменить группу или спе...
1      из чего состоит основное обучение в личном каб...
2      что входит в программу помощи с трудоустройств...
3              подскажите расписание каникул впожалуйста
4      какие инструменты и ресурсы вы предоставляете ...
                             ...                        
295               после чего можно будет получить диплом
296    что сделать чтобы получить итоговый документ о...
297    какие условия получения итогового документа об...
298      какой документ подтверждает уровень компетенций
299    как подтвержрить уровень компетенций после про...
Name: Question, Length: 258, dtype: object

In [ ]:
question[304]

In [120]:
question.to_csv("question.csv",index=False)

In [ ]:
from sklearn.model_selection import train_test_split


X = vect.fit_transform(question)
Y = train_data.answer_class
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', probability=True)
SVM.fit(X_train , y_train)
predictions_SVM = SVM.predict(X_test)

In [ ]:
print("SVM f1 Score -> ",f1_score(predictions_SVM, y_test, average= "micro"))

# ____________________________________________

In [ ]:
from joblib import dump, load
dump(SVM, 'model.joblib') 

In [ ]:
clf = load('/kaggle/working/model.joblib') 


In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer


vect = TfidfVectorizer(max_features=5000,stop_words='english')
X = vect.fit_transform(question)

joblib.dump(vect, 'tfidf_vectorizer.joblib')





In [ ]:
loaded_vectorizer = joblib.load('tfidf_vectorizer.joblib')

question = loaded_vectorizer.transform(["что делать если я хочу изменить группу"])

question

clf.predict_proba(question)

# 2 Вариант

In [122]:
Y = train_data.answer_class

X_train, X_test, y_train, y_test = train_test_split(question,Y,  test_size=0.2, random_state=4)

In [ ]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42, loss = "log_loss"))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

In [ ]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

In [ ]:
from joblib import dump, load
dump(sgd_ppl_clf, 'new_model_2.joblib') 

In [112]:
from joblib import dump, load

model = load('/kaggle/working/new_model_2.joblib') 

def check_predict(model, question):
    arr = []
    arr.append(question)
    if model.predict_proba(arr).max() < 0.1:
        proba = model.predict_proba(arr)
    
        dictionary = {}
        for value in proba:
            for index,j in enumerate(value):
    
                dictionary[j] = index
        
            sort_proba = sorted(value, reverse=True)
        pred_val = sort_proba[:3]
    
    
        predict = []
        for i in pred_val:
            predict.append(int(dictionary[i]))
        return predict
    
    return model.predict(arr)

# print(check_predict(model, "Хочу поговорить с оператором"))
# print(check_predict(model, "че"))
print(check_predict(model, "Хочу поговорить с оператором"))
model.predict(["Хочу поговорить с оператором"])

[-1]


array([-1])

In [ ]:
if model.predict_proba(arr)

In [ ]:
p = sgd_ppl_clf.predict_proba(["Когда будет выдаваться диплом"])
p

In [ ]:
p = sgd_ppl_clf.predict(["то делать, если я хочу изменить группу"])
p

In [ ]:
predicted_sgd = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

# 3 вариант

In [123]:
nlp = spacy.blank("ru")

textcat = nlp.add_pipe("textcat")

In [124]:
textcat.add_label("0")
textcat.add_label("1")
textcat.add_label("2")
textcat.add_label("3")
textcat.add_label("4")
textcat.add_label("5")
textcat.add_label("6")
textcat.add_label("7")
textcat.add_label("8")
textcat.add_label("9")
textcat.add_label("10")
textcat.add_label("11")
textcat.add_label("12")
textcat.add_label("13")
textcat.add_label("14")
textcat.add_label("15")
textcat.add_label("16")
textcat.add_label("17")
textcat.add_label("18")
textcat.add_label("19")
textcat.add_label("20")
textcat.add_label("21")
textcat.add_label("22")
textcat.add_label("23")
textcat.add_label("24")
textcat.add_label("25")
textcat.add_label("26")
textcat.add_label("27")
textcat.add_label("28")
textcat.add_label("29")


1

In [126]:
train_texts = question
train_labels = [{'cats': {
    '0': label == 0,
    '1': label == 1,
    '2': label == 2,
    '3': label == 3,
    '4': label == 4,
    '5': label == 5,
    '6': label == 6,
    '7': label == 7,
    '8': label == 8,
    '9': label == 9,
    '10': label == 10,
    '11': label == 11,
    '12': label == 12,
    '13': label == 13,
    '14': label == 14,
    '15': label == 15,
    '16': label == 16,
    '17': label == 17,
    '18': label == 18,
    '19': label == 19,
    '20': label == 20,
    '21': label == 21,
    '22': label == 22,
    '23': label == 23,
    '24': label == 24,
    '25': label == 25,
    '26': label == 26,
    '27': label == 27,
    '28': label == 28,
    '29': label == 29,
}
                
                } 
                for label in y_train]

In [127]:
train_data = list(zip(X_train, train_labels))
train_data[:3]

[('как проходят занятия на курсе',
  {'cats': {'0': False,
    '1': False,
    '2': False,
    '3': False,
    '4': False,
    '5': False,
    '6': False,
    '7': True,
    '8': False,
    '9': False,
    '10': False,
    '11': False,
    '12': False,
    '13': False,
    '14': False,
    '15': False,
    '16': False,
    '17': False,
    '18': False,
    '19': False,
    '20': False,
    '21': False,
    '22': False,
    '23': False,
    '24': False,
    '25': False,
    '26': False,
    '27': False,
    '28': False,
    '29': False}}),
 ('какие условия и критерии получения удостоверения о повышении квалификации',
  {'cats': {'0': False,
    '1': False,
    '2': False,
    '3': True,
    '4': False,
    '5': False,
    '6': False,
    '7': False,
    '8': False,
    '9': False,
    '10': False,
    '11': False,
    '12': False,
    '13': False,
    '14': False,
    '15': False,
    '16': False,
    '17': False,
    '18': False,
    '19': False,
    '20': False,
    '21': False,
    '

In [128]:
from spacy.util import minibatch
from spacy.training.example import Example

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) 
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

[2024-04-28 00:53:43,730] [INFO] Created vocabulary
[2024-04-28 00:53:43,731] [INFO] Finished initializing nlp object


In [129]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

[2024-04-28 00:54:22,295] [INFO] Created vocabulary
[2024-04-28 00:54:22,296] [INFO] Finished initializing nlp object


{'textcat': 7.4037369126454}
{'textcat': 11.882491929295384}
{'textcat': 13.57193294483285}
{'textcat': 14.274194176104837}
{'textcat': 14.537316263421403}
{'textcat': 14.703749031381543}
{'textcat': 14.896893226776474}
{'textcat': 15.034044587946244}
{'textcat': 15.118210414549917}
{'textcat': 15.195561266716457}


In [142]:
docs = [nlp.tokenizer(text) for text in X_test]

textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

predicted_labels = scores.argmax(axis=1)

pred = []
for label in predicted_labels:
    pred.append(int(textcat.labels[label]))
    

print("spacy f1 Score -> ",f1_score(pred, y_test, average= "micro"))

spacy f1 Score ->  0.7833333333333333


# 4 Вариант

In [148]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
vect = CountVectorizer()



x_train_dtm = vect.fit_transform(X_train)
x_test_dtm = vect.transform(X_test)
rf = RandomForestClassifier(max_depth=10,max_features=10)
rf.fit(x_train_dtm,y_train)
rf_predict = rf.predict(x_test_dtm)

print("spacy f1 Score -> ",f1_score(rf_predict, y_test, average= "micro"))


spacy f1 Score ->  0.7
